In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import Levenshtein
from fuzzywuzzy import process, fuzz

In [2]:
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [3]:
data = pd.read_csv('../data/internships_dataset.csv')
data = data.rename(columns={'Unnamed: 0': 'Index'})
data.set_index("Index",inplace = True)
# data.head()

In [4]:
df = data.copy()

In [5]:
# df.shape

In [6]:
# df.isnull().sum()

In [7]:
# df1 = df[df['Description'].notnull()]

In [8]:
# df1.shape

In [9]:
# df1.head()

In [10]:
# df1.describe()

In [11]:
# df1['Description'] = df1['Job Title'] + df1['Description']

In [12]:
# def preprocess(text):
#     text = text.lower()

#     tokenizer = RegexpTokenizer(r'\w+')
#     text = tokenizer.tokenize(text)
# #     print(text)
# #     text_p = " ".join([char for char in text if char not in string.punctuation])
# #     print(text_p)
# #     text = ' '.join([word for word in text])
# #     print("Text: ",text)
#     stop_words = stopwords.words('english')
#     filtered_words = [word for word in text if word not in stop_words]
    
    
# #     porter = PorterStemmer()
# #     stemmed = [porter.stem(word) for word in filtered_words]
    
#     pos = pos_tag(filtered_words)
#     lemmatizer = WordNetLemmatizer()
#     lem_text =[lemmatizer.lemmatize(word) for word in filtered_words]
    
    
#     return " ".join(word for word in lem_text)

In [13]:
# df1['Description'] = df1['Description'].apply(lambda x: preprocess(x))

In [14]:
# tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
# tfidf_matrix = tf.fit_transform(df1['Description'])

In [15]:
# tfidf_matrix

In [16]:
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [17]:
# cosine_sim.shape

In [18]:
# cosine_sim[0]

In [19]:
# df1 = df1.reset_index()
# Job_Title = df1['Job Title']
# indices = pd.Series(df1.index, index=df1['Job Title'])
# # indices

In [20]:
# df1.skills.head()

In [21]:
# def get_recommendations(title):
#     idx = indices[title]
#     # print (idx)
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     # print (sim_scores)
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     job_indices = [i[0] for i in sim_scores]
#     return Job_Title.iloc[job_indices]

In [22]:
# df['Job Title'].head()[0]

In [23]:
# indices

In [24]:
# get_recommendations("Pre-Silicon Systems Simics developer").head(10)

### Skills based Filtering 

In [25]:
df_skills = df.copy()
# df_skills.head()

In [26]:
df_skills = df_skills[df_skills['title'].notnull()]
# df_skills.shape

In [27]:
def preprocess_skills(text):
    text = text.lower()

    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    stop_words = stopwords.words('english')
    filtered_words = [word for word in text if word not in stop_words]   
    
    return " ".join(word for word in filtered_words)

In [28]:
df_skills['title'] = df_skills['title'].apply(lambda x: preprocess_skills(x))
# df_skills['skills'].head()

In [29]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_skills['title'])

In [30]:
# tfidf_matrix.shape

In [31]:
import pickle
pickle.dump(tfidf_matrix, open('../models/tfidf_vectorizer_title.pickle', 'wb'))

In [32]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [33]:
# cosine_sim.shape,cosine_sim[3]

In [34]:
# df_skills.head()

In [35]:
df_skills = df_skills.reset_index()
skills = df_skills["title"] # + " " + df_skills["Job Title"]
indices = pd.Series(df_skills.index, index = df_skills['title'])
df_skills.to_csv('../data/ints_recommends.csv')
indices.to_csv('../data/title_indices.csv')
skills.to_csv('../data/title.csv')
# indices

In [36]:
# type(skills), type(df_skills), type(indices), type(tfidf_matrix)

In [37]:
# df_skills.skills.head()

In [38]:
def get_most_similar_skills(skills):
    return indices[process.extractOne(skills, indices.index, scorer=fuzz.token_sort_ratio)[0]]

In [39]:
# get_most_similar_skills("linux")

In [40]:
def get_recommendations(input_skills):
#    print(input_skills)
#    idx = indices[input_skills]
#    print(get_most_similar_skills(input_skills.lower()))
    idx = get_most_similar_skills(input_skills.lower())
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    job_indices = [i[0] for i in sim_scores]
    skills = df_skills['title']
    return skills.iloc[job_indices]

In [41]:
# get_recommendations('Java')

In [42]:
# skills

### Deployment

In [50]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import nltk
import string
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from fuzzywuzzy import process, fuzz
import pickle

def get_recommandation(input_skills, tfidf_matrix, df_skills, indices):
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    idx = indices[process.extractOne(input_skills.lower(), indices.index, scorer=fuzz.token_sort_ratio)[0]]
    
    #     print("idx:", idx)
#     print(type(idx))
    
    if isinstance(idx, pd.Series):
#         print("series")
        idx = idx[0]
#         print(idx, type(int(idx)))
    
#     if type(idx) is series:
#         idx = idx[0]
        
#     print(idx)
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    job_indices = [i[0] for i in sim_scores]
    return indices.iloc[job_indices], job_indices

In [44]:
if __name__ == "__main__":
    df_skills = pd.read_csv('../data/title.csv', usecols=['title'])
    main_df = pd.read_csv('../data/ints_recommends.csv', index_col='Index')
    indices = pd.read_csv('../data/title_indices.csv')
    indices = pd.Series(indices.index, index=indices['title'])

    with open("../Models/tfidf_vectorizer_title.pickle", "rb") as f:
        tfidf_matrix = pickle.load(f)
    input_skill = input("Enter the skills ")
    recommandations, job_indices = get_recommandation(input_skill, tfidf_matrix, df_skills, indices)
    # print(recommandations.shape, type(recommandations), recommandations.index.tolist())
    # print(df_skills.iloc[job_indices, :])

Enter the skills java


In [45]:
# main_df.head()

In [46]:
main_df.iloc[job_indices, :].head()

,Unnamed: 0,title,company,location,startdate,duration,stipend,apply,tags,link
Index,,,,,,,,,,
45.0,12543,java intern,Collin IT Solution,20,Office Internships,Pune,Unpaid,https://www.makeintern.com/internships/web-des...,NaN,NaN
162.0,9937,intern,PAJASA Stay Solutions Pvt Ltd,1,Virtual Internships,Mumbai,Rs. 3000-10000,https://www.makeintern.com/internships/marketi...,NaN,NaN
246.0,10021,intern,Euphoric Beverages,1,Office Internships,Bangalore,Rs. 5000-8000,https://www.makeintern.com/internships/marketi...,NaN,NaN
96.0,2003,java development,KaHa Technologies,Bangalore,Starts immediatelyImmediately,4 Months,12000-15000 /month,18 Feb' 22,Internship \n\n ...,https://internshala.com/internships/computer%2...
125.0,2032,java development,Liberin Technologies Private Limited,Noida,Starts immediatelyImmediately,6 Months,5000 /month,18 Feb' 22,Internship with job offer \n\n ...,https://internshala.com/internships/computer%2...


In [47]:
# temp = df_skills.iloc[job_indices, :]
# temp.head()

In [48]:
# indices = pd.read_csv('../Data/skills_indices.csv')
# indices = pd.Series(indices.index, index=indices['skills'])
# type(indices), indices

In [49]:
# pd.read_csv('../Data/skills.csv', use_cols)